In [ ]:
import serial.tools.list_ports

def detect_arduino():
    ports = list(serial.tools.list_ports.comports())
    for port in ports:
        if "Arduino" in port.description or "ttyACM" in port.device:
            return port.device
    raise Exception("Arduino non détecté. Veuillez vérifier la connexion.")

In [ ]:
def connexion_arduino():
    global ser
    
    # Détecter automatiquement le port série de l'Arduino
    port = detect_arduino()
    baudrate = 9600  # Assurez-vous que la vitesse correspond à celle configurée sur la carte Arduino
    
    # Ouvrir la connexion série
    ser = serial.Serial(port, baudrate)
    
    # Attendre un court instant pour permettre la connexion
    sleep(2)

In [9]:
def reset_arduino():
    ser.reset_input_buffer()
    # Envoyer la commande de reset
    ser.setDTR(False)    # DTR (Data Terminal Ready) à l'état bas pendant un court instant
    sleep(0.1)
    ser.reset_input_buffer()
    #ser.flushInput()    # reinitialisation des données
    ser.setDTR(True)    # DTR à l'état haut pour terminer le reset

In [10]:
def mesure_unique():
    tensions = []
    for _ in range(10) :
        ligne = ser.readline().decode('utf-8').strip()
        if ligne and ',' in ligne:
            try:
                temps, tension = ligne.split(',')
                float(temps)  # Vérifie si le temps est un nombre valide
                tension = float(tension)  # Vérifie si la tension est un nombre valide
                tensions.append(tension)
            except ValueError:
                continue  # Si la conversion échoue, on passe à la ligne suivante
    ## on élimine les valeurs aberrantes par la méthode IQR :
    # Calculer le premier quartile (25ème percentile)
    Q1 = percentile(tensions, 25)
    
    # Calculer le troisième quartile (75ème percentile)
    Q3 = percentile(tensions, 75)
    
    # Calculer l'IQR
    IQR = Q3 - Q1
    
    # Définir les limites inférieure et supérieure pour les valeurs aberrantes
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Supprimer les valeurs aberrantes
    tensions_non_aberrantes = [value for value in tensions if lower_bound <= value <= upper_bound]

    return mean(tensions_non_aberrantes)

In [11]:
def lire_ligne_valide():
    while True:
        ligne = ser.readline().decode('utf-8').strip()
        if ligne and ',' in ligne:
            try :
                temps, tension = ligne.split(',')
                float(temps)  # Vérifie si le temps est un nombre valide
                float(tension)  # Vérifie si la tension est un nombre valide
                return temps, ligne
            except ValueError:
                continue  # Si la conversion échoue, on passe à la ligne suivante


In [12]:
def acquisition_donnees(duree):
    donnees = "t,U"
    
    while True :
        try:
            temps, ligne = lire_ligne_valide()
            donnees += f"\n{ligne}"
            if temps > duree :
                break
        except KeyboardInterrupt:
            # Interrompre la boucle si l'utilisateur appuie sur le bouton stop
            break
    print(donnees)
    # Fermer la connexion série
    fichier = open("arduino.csv", 'w')
    fichier.write(donnees)
    fichier.close()
    ser.close()

In [ ]:
def update_graph():
    global t, u, line, ax, fig, temps_initial, last_update_time, interval
    try:
        ligne = ser.readline().decode('utf-8').strip()
        print(f"Ligne reçue : {ligne}")  # Impression de débogage
        if not ligne:
            return
        
        if ',' not in ligne:
            print(f"Ligne invalide reçue : {ligne}")
            return
        
        valeurs = ligne.split(',')
        if len(valeurs) != 2:
            print(f"Format invalide : {ligne}")
            return
        
        temps, tension = valeurs
        temps = float(temps) * 1e-3  # Conversion en secondes
        tension = float(tension)
        
        if temps_initial is None:
            temps_initial = temps
        
        temps_relatif = temps - temps_initial
        
        # Échantillonner les points de manière uniforme
        if temps_relatif >= last_update_time + interval:
            t.append(temps_relatif)
            u.append(tension)
            last_update_time += interval
            
            line.set_data(t, u)
            
            clear_output(wait=True)
            display(fig)
    except Exception as e:
        print(f"Erreur lors de la mise à jour du graphique : {e}")



In [ ]:
def trace_temps_reel(duree):
    global t, u, temps_initial, last_update_time, interval
    ser.reset_input_buffer()
    t = []
    u = []
    temps_initial = None
    last_update_time = 0
    interval = duree / 100  # Intervalle pour échantillonner les points
    
    # Boucle de préchauffage pour lire et ignorer quelques lignes initiales
    for _ in range(10):
        ser.readline()
    
    # Ajouter une mesure initiale pour t=0
    #update_graph()
    
    while True:
        update_graph()
        plt.pause(0.001)  # Réduire davantage la durée de la pause pour augmenter la fréquence de lecture
        
        if t and t[-1] >= duree:
            break
    
    ser.close()